# Analyzing Mobile App Data

The goal of this project is to help our developers understand which types of apps are likely to attract more users on Google Play and the App Store

In [2]:
def explore_data(dataset, start, end, rows_and_columns = False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

                                                The explore_data() function does the following:

## Parameters:
- Dataset, which will be a list of lists
- Start and end, which will both be integers and represent the starting and the ending indices of a slice from the dataset.
- rows_and_columns, which will be a Boolean and has False as a default argument

## Functionality:
- Slices the dataset using `dataset[start:end]`
- Loops through the slice, and for each iteration, prints a row and adds a new line after that row using `print('\n')`
  
 * `The \n in print('\n')` is a special character that won't print. Instead, the `\n` character adds a new line, and we use `print('\n')` to add some blank space between rows.

## Optional Row and Column Count:
- If `rows_and_columns` is set to `True`, it prints the number of rows and columns in the dataset.

  
## The dataset shouldn't have a header row, or the function will print the wrong number of rows (one more row compared to the actual length)

## With the code below we are opening both iOS and Google Store data:

In [5]:
from csv import reader

## Google Play ###
opened_file = open('googleplaystore.csv', encoding='utf-8')
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android = android[1:]

### App Store ###

opened_file = open('AppleStore.csv', encoding='utf-8')
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

In [6]:
print(android_header)
print('\n')
explore_data(android, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


# The next stage is to clean the data:
- Remove malfunctioned data
- Remove duplicates
- Remove non-English apps
- Remove apps that aren't free

In [8]:
del android[10472]

## `del android[10472]` was used to delete a row with corrupted data(missing information) and that WILL brick our code below.

In [10]:
uniq_apps = []
duplicate_apps = []

for app in android:
    name = app[0]
    if name in uniq_apps:
        duplicate_apps.append(name)
    else:
        uniq_apps.append(name)

print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:15])

Number of duplicate apps: 1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


As we can see from the row above, we have multiple duplicates available and the main difference is the amount of reviews.
So likely we want to use the one with the biggest amount of reviews available (since it would mean the latest timestamp taken).

Further steps:
- Create a separate dictionary out of the current one using a unique app name and the highest number of reviews (latest timestamp).

In [12]:
reviews_max = {}

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

print('Expected length:', len(android) - 1181)
print('Actual length:', len(reviews_max))

Expected length: 9659
Actual length: 9659


The code above shows us the amount of actual apps that we should have in our list. We don't want to count certain apps
that are already a part of our list.

In [14]:
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])

    if (n_reviews == reviews_max[name]) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name)

explore_data(android_clean, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


# Now we have made an actual list with no duplicate data.

For the duplicate cases, we'll only keep the entries with the highest number of reviews.

We use English for the apps we develop at our company, and we'd like to analyze only the apps that are designed for an English-speaking audience.

Due to that next stage will be to remove apps that are non English.

In [17]:
def name_check(string):
    non_ascii_count = 0

    for char in string:
        if ord(char) > 127:
            non_ascii_count +=1

            if non_ascii_count > 3:
                return False

    return True

The function above will check:

- How many characters from non-ASCII does the app name have. If it has non-ASCII characters and there is more than 3 we will treat this as non English App

In [19]:
english_apps_android = []
english_apps_ios = []

for app in android_clean:
    eng = app[0]
    if name_check(eng):
        english_apps_android.append(app)

for app in ios:
    eng_ios = app[1]
    if name_check(eng_ios):
        english_apps_ios.append(app)

print(english_apps_android[:1], len(english_apps_android))
print(english_apps_ios[:1], len(english_apps_ios))

[['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']] 9614
[['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']] 6183


We have cleaned the data from non-ascii characters and now we have:

- 9614 Apps in Android - `english_apps_android`
- 6183 Apps in iOS - `english_apps_ios`

## So far we have:

- Removed inaccurate data
- Removed duplicates
- Removed non-English apps

The final step is to remove paid apps.

In [33]:
free_apps_android = []
free_apps_ios = []

for app in english_apps_android:
    if app[7] == '0' or app[7] == '0.0' or app[7] == 'Free':
        free_apps_android.append(app)

for app in english_apps_ios:
    if app[4] == '0' or app[4] == 'Free' or app[4] == '0.0':
        free_apps_ios.append(app)

print(len(free_apps_android))
print(len(free_apps_ios))

8864
3222


## Capstone: Data was cleaned

After all cleanings we have:

- 8864 Android Apps : `free_apps_android`
- 3222 iOS Apps : `free_apps_ios`

The next part is to find an app that will be profitable in both stores. 
Since we are using free apps and our revenue comes from the ads, we need to find the most popular and most downloaded apps.

We can start by analysing and determining common genres for both markets. 

- We will be focusing on Genre & Category for Android
- And prime_genre for iOS store

In [70]:
def freq_table(dataset, index):
    table = {}
    count = 0
    
    for app in dataset:
        count += 1
        value = app[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    percentages = {}
    for key in table:
        percentage = (table[key] / count) * 100
        percentages[key] = percentage
    return percentages

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

print(display_table(free_apps_android, 1))
print('\n')
print(display_table(free_apps_android, 9))
print('\n')
print(display_table(free_apps_ios, 11))

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

## Functions above do 2 things:

- Allow us to generate a frequency table with parameters `dataset`, `index` plus provides us with percentage of occurence
- Second function transforms the frequency table into a list of tuples and then sorts the list in descending order

## Percentage and result that we see:

Based on the data we have, the most popular apps for the English Android Store have categories and genres:

Category based:

- Family 18.90%
- Games 9.72%
- Tools 8.46%
- Business 4.59%

Genre based:

- Tools 8.44%
- Entertainment 6.06%
- Education 5.34%
- Business 4.59%

As for the iOS store, based on prime_genre:

- Games 58.16%
- Entertainment 7.88%
- Photo & Video 4.96%
- Education 3.66%

We can see that more than half of the apps in iOS store (58.16%) are Games, while GooglePlay Market shows that games have only 9.72% which is top2 result, while the Family Category has 18.90%. But if we check what the actual Family category means, there are mostly games for kids.

In [80]:
genres_ios = freq_table(free_apps_ios, 11)

for genre in genres_ios:
    total = 0
    len_genre = 0
    for app in free_apps_ios:
        genre_app = app[11]
        if genre_app == genre:
            user_ratings = float(app[5])
            total += user_ratings
            len_genre += 1
    avg_number = total / len_genre
    print(genre, ':', avg_number)

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22788.6696905016
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


- Calculation above helps us to calculate the average number of installs for each app genre.
- Since App Store information is missing, we have used for it `rating_count_tot` which is total number of user ratings

## Based on the data we have, the most popular ones are:
- Navigation
- Reference
- Social Networking
- Music

Some of the genres might be influenced by popular apps (e.g Waze/Google Maps) in Navigation ; Bible in Reference and etc.

In [95]:
genres_android = freq_table(free_apps_android, 1)

for app in genres_android:
    total = 0
    len_category = 0
    for row in free_apps_android:
        category_app = row[1]
        if category_app == app:
            installs = row[5]
            installs = installs.replace('+', '')
            installs = installs.replace(',', '')
            installs = float(installs)
            total += installs
            len_category += 1
    avg_num = total / len_category
    print(app, ':', avg_num)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_